## Creating a Song Recommender Algorithm


This song recommender acts as follows:



- User Gives the Programme a Song and Artist

    If the song is in the current or all time top 200 list, It will recommend another song from that list.
    
    If the song is **NOT** in the above list, it will search for the song on spotify, extract the features, and recommend another, similar song, providing the spotify link so that the user can listen to it. 
    
    
    
---    
 

In [3]:
def song_recommender():


    ################################## IMPORT LIBRARIES ##################################    
    
    import spotipy
    from spotipy.oauth2 import SpotifyClientCredentials
    import pandas as pd   
    import pickle
    
    
    ################################## IMPORT NECESSARY FILES ##################################
    

    
    top = pd.read_csv('top.csv')
    spot = pd.read_csv('spotify_songs_clusters.csv')
    secrets_file = open("../secrets.txt","r")
    string = secrets_file.read()
    secrets_dict={}
    
    for line in string.split('\n'):     
        if len(line) > 0:
            secrets_dict[line.split(':')[0]]=line.split(':')[1]
    
    #InitializeSpotiPy with user credentials
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))
    
    
    
    
    
    
    ################################## TAKE SONG AND ARTIST NAME ##################################

    song = input("write the name of a song: ") 
    song = song.lower()
    artist = input('write the name of an artist: ')
    artist = artist.lower()

    ################################## IF SONG IN TOP 100 ##################################
    if (song in list(top['track']))&(artist in list(top['artist'])):
        print(f"HOT HOT HOT!, we also recommend: %s, by %s." %(top['track'][num],top['track'][num]))    
    
    
    ################################## IF NOT IN TOP 100 ##################################
    elif song not in list(top['track']):
    
    
    
    ################################## CONNECT TO API ##################################
    
                
                    
            
        ################################## SEARCH FOR SONG ##################################
        
        
        
        results = sp.search(q= artist + song, limit=10)
        
        
        
        
        ################################## EXTRACT & TRANSFORM FEATURES ##################################

        
        #Loading the model and scalers from the file using pickle

        KM = pickle.load(open('models_scalers/kmeans.pkl','rb'))
        scaler = pickle.load(open('models_scalers/scaler.pkl','rb'))        
        
        
        #EXTRACT
        mini_df = pd.DataFrame({'features': sp.audio_features(results['tracks']['items'][0]['uri'])})
        
        
        #NORMALISE
        features = json_normalize(mini_df['features']).drop(columns= ['key', 'uri', 'duration_ms','mode', 'time_signature', 'type', 'id', 'track_href', 'analysis_url'], axis=1)
        
        
        #SCALE
        features_scaled = scaler.transform(features)
        
        
        
        ################################## FIND CLUSTER ##################################
        
               
        
        #DETERMINE CLUSTER NUMBER
        
        
        clusternum = KM.predict(features_scaled)[0]
        
        #FIND RANDOM SONG IN FILTERED DATAFRAME BY CLUSTER NUMBER 
        clusdf = spot[spot['cluster'] == clusternum].reset_index(drop = True).iloc[randint(0,len(spot[spot['cluster'] == clusternum])-1)]
        
        
        
        
        ################################## OUTPUT THE SONG ##################################
        
        
        #SEARCH FOR SONG IN SPOTIFY
        outresults = sp.search(q= clusdf[0] + clusdf[1], limit=10)
        
        
        
        #RECOMMEND SONG, ARTIST, AND PROVIDE LINK
        print("I recommend %s, by %s. Follow: %s to listen to the song" %(outresults['tracks']['items'][0]['name'], outresults['tracks']['items'][0]['album']['artists'][0]['name'],outresults['tracks']['items'][0]['external_urls']['spotify']))
        
        #print('I recommend "'+outresults['tracks']['items'][0]['name']+'", by', outresults['tracks']['items'][0]['album']['artists'][0]['name']+". Follow:", outresults['tracks']['items'][0]['external_urls']['spotify'],"to listen to the song")
        

In [4]:
song_recommender()

write the name of a song: Hello
write the name of an artist: Lionel Richie


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=0mHyWYXmmCB9iQyK18m3FQ:
 Max Retries, reason: too many 429 error responses